In [1]:
!pip install keybert
!pip install --user flair

  Using cached torch-2.0.1-cp38-cp38-win_amd64.whl (172.4 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0
    Uninstalling torch-2.0.0:
      Successfully uninstalled torch-2.0.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformer-smaller-training-vocab 0.3.2 requires torch!=2.0.1,<3.0.0,>=1.8.0, but you have torch 2.0.1 which is incompatible.


  Using cached torch-2.0.0-cp38-cp38-win_amd64.whl (172.3 MB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.15.2 requires torch==2.0.1, but you have torch 2.0.0 which is incompatible.


In [3]:
import numpy as np
import pandas as pd
import time
import random
from keybert import KeyBERT
from flair.embeddings import TransformerDocumentEmbeddings

In [48]:
data = pd.read_csv("./fivecart_total_pre.csv", encoding='utf-8')
data.head(5)

,report_writer,date_writer,title,book_writer,book_company,book_date,report_text,is_classic
0,조장환,2023/09/21,노르웨이의 숲,무라카미 하루키,민음사,2013/09/02,"""그때 나는 보잉 747기기 좌석에 앉아있었다."" 1982년에 자신을 회...",NaN
1,갱근,2023/09/20,역행자 (돈·시간·운명으로부터 완전한 자유를 얻는 7단계 인생 공략집),자청,웅진지식하우스,2022/05/30,이 책은 제가 경제에 대해 아는것이 없다보니 알아보려고 이것저것 찾아보다가제가...,NaN
2,예봄이,2023/09/20,덧니가 보고 싶어 (정세랑 장편소설),정세랑,난다,2019/11/05,개강이 실감 나지 않았던 지난 3주를 흘려보내고 나니 인제야 학교 다니는 일상...,NaN
3,떠다니는 윤슬,2023/09/20,"파친코 1~2권 세트 (전2권, 개정판│애플TV 드라마 '24 May 2018파친코...",이민진,인플루엔셜,2022/08/25,파친코는 일본의 도박성 대중오락이다. 파친코라는 이름은 게임할 때 생기는 마...,NaN
4,아우룸,2023/09/20,생의 한가운데,루이제 린저,문예출판사,1998/01/20,생을 다루는 예술 작품은 참으로 많다. 톨스토이의 [이반 일리치의 죽음]이 ...,NaN


In [52]:
data

,report_writer,date_writer,title,book_writer,book_company,book_date,report_text,is_classic
0,조장환,2023/09/21,노르웨이의 숲,무라카미 하루키,민음사,2013/09/02,"""그때 나는 보잉 747기기 좌석에 앉아있었다."" 1982년에 자신을 회...",NaN
1,갱근,2023/09/20,역행자 (돈·시간·운명으로부터 완전한 자유를 얻는 7단계 인생 공략집),자청,웅진지식하우스,2022/05/30,이 책은 제가 경제에 대해 아는것이 없다보니 알아보려고 이것저것 찾아보다가제가...,NaN
2,예봄이,2023/09/20,덧니가 보고 싶어 (정세랑 장편소설),정세랑,난다,2019/11/05,개강이 실감 나지 않았던 지난 3주를 흘려보내고 나니 인제야 학교 다니는 일상...,NaN
3,떠다니는 윤슬,2023/09/20,"파친코 1~2권 세트 (전2권, 개정판│애플TV 드라마 '24 May 2018파친코...",이민진,인플루엔셜,2022/08/25,파친코는 일본의 도박성 대중오락이다. 파친코라는 이름은 게임할 때 생기는 마...,NaN
4,아우룸,2023/09/20,생의 한가운데,루이제 린저,문예출판사,1998/01/20,생을 다루는 예술 작품은 참으로 많다. 톨스토이의 [이반 일리치의 죽음]이 ...,NaN
...,...,...,...,...,...,...,...,...
21862,icaros,2009/11/02,사치의 나라 럭셔리 코리아,김난도,미래의창,2007/03/29,사치의 나라 럭셔리 코리아 김남도 지음 출판사 : 미래의 창 2007년 ...,NaN
21863,icaros,2009/11/02,렉스,캐슬린 루이스,Human&Books,2008/12/05,"렉스 / 한 서번트 이야기 / REX : A Mother, Her Autist...",NaN
21864,icaros,2009/11/02,나의 아들 달라이라마,캐둡된돕,한언,2000/11/25,나의 아들 달라이라마 ( DALAI LAMA MY SON ) 캐둡된둡 ...,NaN
21865,불산,2009/11/02,바리데기,황석영,창비,2007/07/13,바리데기 -분쟁과 굶주림 속에 고통받는 모든 이들을 위한 굿- 아직 한 ...,NaN


In [53]:
doc = data['report_text'][4]
doc

'    생을 다루는 예술 작품은 참으로 많다. 톨스토이의 [이반 일리치의 죽음]이 가장 심도깊게 생의 의미를 다룬 작품이라 할 수 있고, 이외에도 무라카미 하루키의 [노르웨이의 숲], 양귀자의 [모순], 헤르만 헤세의 [싯다르타]나 [데미안], 로맹 가리의 [자기 앞의 생], 좀 변칙적이긴 하나 아인 랜드의 [파운틴헤드] 등을 삶의 의미를 파헤치는 문학 작품의 예시로 들 수 있을 것이다. 애초에 예술이란 것이 삶의 어느 구석에서든 그 의미가 무엇인가를 고찰하는 것과 절대로 무관할 수는 없는 노릇이기에 모든 예술 작품은 곧 삶을 되새기는 것이라고 요약할 수 있겠으나, 그중에서도 위에 언급한 소설들처럼 직접적으로 생을 중심 주제로 제시하고 전개해나가는 부류들은 나름의 장르를 형성하고 있다고 볼 수 있다. 이들 소설은 기본적으로 생애라는 것에서 인간이 느끼는 방황을 제시하고, 그 방황에 해답은 없다고 결론 내리는 작품도 있고 방황에 굴복하여 패배감이 스며든 해답을 내리는 작품도 있고 방황을 극복하고 때려잡을 작가 고유의 완전한 해답을 제시하는 작품도 있다. <생의 한가운데>는 이중에서 일종의 굴복적인 해답을 제시하는 갈래로 분류될 수 있을 것이다.   이 소설은 니나라는 중심 주인공이 있고 그녀를 관찰하는 언니의 시점에서 줄거리가 전개된다. 니나가 작중 시점 37살에 이르기까지의 인생 항로를 그녀가 10대일 때부터 긴밀한 관계를 맺어온 슈타인이라는 약 20살 연상의 남자의 일기를 통해 언니는 훑게 되는데, 중간중간 니나가 쓴 자전적 소설이 삽입되어 그녀의 인생을 빗대어 말해주기도 하고 슈타인의 일기뿐만 아니라 니나의 편지같은 다른 글 형식으로도 니나라는 인물의 삶과 인간상을 형성한다. 그럼으로써 소설 내내 드러나는 니나의 두드러진 특성이 나오는데, 니나는 근본적으로 인간의 부유하는 욕망을 모조리 긍정하려는 인간이라는 점이다. 인간은 참으로 다방면의 욕구를 가지고 있고 많은 경우에 서로 모순되고 상충되는 욕망들까지 동시에 여러개를 지닐 수가 있는 존재다. 간단하게는

In [42]:
len(data['report_text'][4])

2726

In [ ]:
#기본 키버트 라이브러리 제공 모델로 시도, unigram, bigram, trigram

In [6]:
import joblib

In [ ]:
modellist = ['all-mpnet-base-v2', 'distiluse-base-multilingual-cased-v1', 'klue/roberta-base', 'klue/roberta-small', 'klue/roberta-large', 'klue/bert-base']

In [ ]:
#모델 저장

In [8]:
kw_model = KeyBERT()

In [9]:
joblib.dump(kw_model, './kw_model_base.pkl')

['./kw_model_base.pkl']

In [17]:
kw_model_0 = KeyBERT(modellist[0])
joblib.dump(kw_model_0, './kw_model_all-mpnet-base-v2.pkl')

['./kw_model_all-mpnet-base-v2.pkl']

In [18]:
kw_model_1 = KeyBERT(modellist[1])
joblib.dump(kw_model_1, './kw_model_distiluse-base-multilingual-cased-v1.pkl')

['./kw_model_distiluse-base-multilingual-cased-v1.pkl']

In [21]:
kw_model_2 = KeyBERT(TransformerDocumentEmbeddings(modellist[2]))
joblib.dump(kw_model_2, './kw_model_klue_roberta-base.pkl')

['./kw_model_klue_roberta-base.pkl']

In [22]:
kw_model_3 = KeyBERT(TransformerDocumentEmbeddings(modellist[3]))
joblib.dump(kw_model_3, './kw_model_klue_roberta-small.pkl')

['./kw_model_klue_roberta-small.pkl']

In [23]:
kw_model_4 = KeyBERT(TransformerDocumentEmbeddings(modellist[4]))
joblib.dump(kw_model_4, './kw_model_klue_roberta-large.pkl')

C:\Users\liber\anaconda3\envs\fiveCart\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\liber\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


['./kw_model_klue_roberta-large.pkl']

In [24]:
kw_model_5 = KeyBERT(TransformerDocumentEmbeddings(modellist[5]))
joblib.dump(kw_model_4, './kw_model_klue_bert-base.pkl')

['./kw_model_klue_bert-base.pkl']

In [10]:
loaded_model = joblib.load('./kw_model_base.pkl')

In [25]:
%%time
kw_model = loaded_model
keywords_1 = kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 1), stop_words=None)
keywords_2 = kw_model.extract_keywords(doc, keyphrase_ngram_range=(2, 2), stop_words=None)
keywords_3 = kw_model.extract_keywords(doc, keyphrase_ngram_range=(3, 3), stop_words=None)

CPU times: total: 13.4 s
Wall time: 9.33 s


In [14]:
%%time
kw_model = KeyBERT()
keywords_1 = kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 1), stop_words=None)
keywords_2 = kw_model.extract_keywords(doc, keyphrase_ngram_range=(2, 2), stop_words=None)
keywords_3 = kw_model.extract_keywords(doc, keyphrase_ngram_range=(3, 3), stop_words=None)

CPU times: total: 15.8 s
Wall time: 10.9 s


In [26]:
print(keywords_1)
print(keywords_2)
print(keywords_3)

[('이성의', 0.506), ('일리치의', 0.5), ('인물의', 0.4869), ('다방면의', 0.4782), ('일종의', 0.4704)]
[('인간 이성의', 0.5413), ('인간의 극복과', 0.5398), ('인식한다 인간의', 0.5259), ('구석에서든 의미가', 0.5255), ('종국에 인간의', 0.5162)]
[('인간은 참으로 다방면의', 0.5761), ('데미안 로맹 가리의', 0.5497), ('형식으로도 니나라는 인물의', 0.549), ('생의 한가운데 문체는', 0.5473), ('생의 의미를 다룬', 0.5458)]


In [ ]:
#sbert-net performance sentence embedding 가장 높은 모델로 시도, unigram, bigram, trigram

In [19]:
%%time
modelname = 'all-mpnet-base-v2'
kw_model = KeyBERT(modelname)
keywords_1 = kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 1), stop_words=None)
keywords_2 = kw_model.extract_keywords(doc, keyphrase_ngram_range=(2, 2), stop_words=None)
keywords_3 = kw_model.extract_keywords(doc, keyphrase_ngram_range=(3, 3), stop_words=None)

CPU times: total: 1min 27s
Wall time: 48.9 s


In [20]:
print(keywords_1)
print(keywords_2)
print(keywords_3)

[('구석에서든', 0.7151), ('순간순간의', 0.6937), ('부산물들을', 0.6759), ('굴복적인', 0.6678), ('구석은', 0.6659)]
[('종국에 인간의', 0.7426), ('자신의 순간순간의', 0.7413), ('한가운데 문체는', 0.7407), ('구석에서든 의미가', 0.7394), ('한가운데 이중에서', 0.737)]
[('종국에 인간의 극복과', 0.7548), ('나는 종국에 인간의', 0.7532), ('그녀가 자신의 순간순간의', 0.7488), ('무언가를 선택하는 것은', 0.7462), ('순간순간의 충동에 맞서게', 0.7453)]


In [ ]:
#sbert-net 다국어 모델로 시도

In [21]:
%%time
modelname = 'distiluse-base-multilingual-cased-v1'
kw_model = KeyBERT(modelname)
keywords_1 = kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 1), stop_words=None)
keywords_2 = kw_model.extract_keywords(doc, keyphrase_ngram_range=(2, 2), stop_words=None)
keywords_3 = kw_model.extract_keywords(doc, keyphrase_ngram_range=(3, 3), stop_words=None)

CPU times: total: 22.1 s
Wall time: 12.1 s


In [22]:
print(keywords_1)
print(keywords_2)
print(keywords_3)

[('예술이란', 0.3257), ('작품의', 0.3041), ('작품이라', 0.2935), ('예술', 0.2833), ('작품은', 0.2721)]
[('예술 작품은', 0.3762), ('애초에 예술이란', 0.3645), ('다룬 작품이라', 0.3627), ('다루는 예술', 0.3605), ('예술이란 것이', 0.3405)]
[('생을 다루는 예술', 0.4942), ('다루는 예술 작품은', 0.4223), ('작품도 있다 생의', 0.4176), ('예술 작품은 참으로', 0.407), ('예술 작품은 삶을', 0.3922)]


In [ ]:
#klue/roberta-base 모델로 시도

In [25]:
roberta_b = TransformerDocumentEmbeddings('klue/roberta-base')

In [26]:
%%time
kw_model = KeyBERT(model=roberta_b)
keywords_1 = kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 1), stop_words=None)
keywords_2 = kw_model.extract_keywords(doc, keyphrase_ngram_range=(2, 2), stop_words=None)
keywords_3 = kw_model.extract_keywords(doc, keyphrase_ngram_range=(3, 3), stop_words=None)

CPU times: total: 1min 53s
Wall time: 1min 7s


In [27]:
print(keywords_1)
print(keywords_2)
print(keywords_3)

[('점이다', 0.9655), ('일기뿐만', 0.9653), ('작품이라', 0.9649), ('것은', 0.9646), ('것이다', 0.9644)]
[('것이다 결론적으로', 0.9715), ('때문이다 이런', 0.9686), ('살아가기에 니나의', 0.967), ('있다 이들', 0.9669), ('슈타인의 일기뿐만', 0.9667)]
[('것이다 결론적으로 작가는', 0.9708), ('때문이다 이런 태도가', 0.9699), ('행각이었을 것이다 결론적으로', 0.9695), ('것이다 소설은 니나라는', 0.9689), ('때문이다 허나 니나의', 0.9687)]


In [ ]:
#klue/roberta-small 모델로 시도

In [14]:
roberta_s = TransformerDocumentEmbeddings('klue/roberta-small')

In [29]:
%%time
kw_model = KeyBERT(model=roberta_s)
keywords_1 = kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 1), stop_words=None)
keywords_2 = kw_model.extract_keywords(doc, keyphrase_ngram_range=(2, 2), stop_words=None)
keywords_3 = kw_model.extract_keywords(doc, keyphrase_ngram_range=(3, 3), stop_words=None)

CPU times: total: 1min 11s
Wall time: 40.1 s


In [30]:
print(keywords_1)
print(keywords_2)
print(keywords_3)

[('예술이란', 0.789), ('있어서는', 0.787), ('변칙적이긴', 0.7827), ('그중에서도', 0.7827), ('형식으로도', 0.7797)]
[('니나의 편지같은', 0.8232), ('인간상을 형성한다', 0.8228), ('방법임을 역설한다', 0.8212), ('집념어린 추구는', 0.8181), ('형식으로도 니나라는', 0.8159)]
[('삶과 인간상을 형성한다', 0.847), ('소설은 기본적으로 생애라는', 0.8468), ('소설들처럼 직접적으로 생을', 0.842), ('대한 집념어린 추구는', 0.8405), ('진실된 자세였음을 이해한다', 0.8399)]


In [ ]:
#klue/klue/bert-base 모델로 시도

In [31]:
bert_b = TransformerDocumentEmbeddings('klue/bert-base')

In [32]:
%%time
kw_model = KeyBERT(model=bert_b)
keywords_1 = kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 1), stop_words=None)
keywords_2 = kw_model.extract_keywords(doc, keyphrase_ngram_range=(2, 2), stop_words=None)
keywords_3 = kw_model.extract_keywords(doc, keyphrase_ngram_range=(3, 3), stop_words=None)

CPU times: total: 2min 24s
Wall time: 1min 32s


In [27]:
loaded_model_roberta_large = joblib.load('./kw_model_klue_roberta-large.pkl')

In [28]:
%%time
kw_model = loaded_model_roberta_large
keywords_1 = kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 1), stop_words=None)
keywords_2 = kw_model.extract_keywords(doc, keyphrase_ngram_range=(2, 2), stop_words=None)
keywords_3 = kw_model.extract_keywords(doc, keyphrase_ngram_range=(3, 3), stop_words=None)

CPU times: total: 9min 17s
Wall time: 5min 28s


In [29]:
print(keywords_1)
print(keywords_2)
print(keywords_3)

[('37살에', 0.9653), ('슈타인이라는', 0.9648), ('격정적이고', 0.9646), ('변칙적이긴', 0.9643), ('소설들처럼', 0.9636)]
[('점이다 인간은', 0.969), ('것이다 결론적으로', 0.9687), ('것이다 애초에', 0.9685), ('슈타인이라는 20살', 0.9667), ('시점 37살에', 0.9661)]
[('것이다 애초에 예술이란', 0.9734), ('인간이라는 점이다 인간은', 0.9713), ('행각이었을 것이다 결론적으로', 0.9713), ('있을 것이다 소설은', 0.9708), ('때문이다 이런 태도가', 0.9686)]


In [36]:
def cos_value_mean(keywords):
    res = 0
    for i in keywords:
        res += i[1]
    return res/len(keywords)

In [39]:
cos_value_mean(keywords_3)

0.97108

In [66]:
def random_doc_ext(data):
    cnt = len(data)
    rand_list = []
    to10 = 0
    while to10<10:
        n = random.randrange(0, cnt)
        if(n in rand_list):
            continue
        else:
            rand_list.append(n)
            to10+=1     
    doc_list = [[i for j in range(2)] for i in rand_list]
    j=0
    for i in rand_list:
        doc_list[j][1] = data.loc[i,'report_text']
        j+=1
    return doc_list

In [44]:
def keyBERT_model(modelname, doc):
    start = time.time()
    kw_model = KeyBERT(model=modelname)
    keywords_1 = kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 1), stop_words=None)
    keywords_2 = kw_model.extract_keywords(doc, keyphrase_ngram_range=(2, 2), stop_words=None)
    keywords_3 = kw_model.extract_keywords(doc, keyphrase_ngram_range=(3, 3), stop_words=None)
    print(keywords_1)
    print(keywords_2)
    print(keywords_3)
    print(f"{time.time()-start:.4f} sec") # 수행시간 측정

In [108]:
def keyBERT_model_whole(model, doc):
    start = time.time()
    kw_model = model
    keywords_1 = kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 1), stop_words=None)
    keywords_2 = kw_model.extract_keywords(doc, keyphrase_ngram_range=(2, 2), stop_words=None)
    keywords_3 = kw_model.extract_keywords(doc, keyphrase_ngram_range=(3, 3), stop_words=None)
    print(keywords_1)
    print("keywords_1:", cos_value_mean(keywords_1))
    print(keywords_2)
    print("keywords_2:", cos_value_mean(keywords_2))
    print(keywords_3)
    print("keywords_3:", cos_value_mean(keywords_3))
    print(f"{time.time()-start:.4f} sec") # 수행시간 측정

In [ ]:
##랜덤 문서 추출

In [67]:
doc_list = random_doc_ext(data)

In [68]:
doc_list

[[8385,
  '    단순한 암기보다는 전후 관계를 파악하거나 왜 그럴 수밖에 없는 지를 논리적으로 따져보며 공부했을 때 보다 명료하고 오래 기억을 할 수 있다. 더 나아가면 아직 배우지 않은 부분에 대해서 예측을 할 수 있는 수준까지 있는데, 특히 역사부문에서 두드러지게 나타나는 특성이다. 우리는 흔히 ‘역사를 통해 배운다.’ 라는 말을 하고는 하는데 오늘날까지의 발자취를 돌아보며 현재를 이해하고, 또 미래까지 엿볼 수 있다는 기대감이 깃들어있다.     우리가 가장 궁금해 하는 미래는 무엇보다 우리 자신이기 때문에 인류의 역사는 언제나 관심의 대상이었다. 그러나 우리는 인간 그 자체보다는 문명의 발전에 치우쳐 공부한 경향이 있다. 농업혁명, 과학혁명, 산업혁명 그리고 정보혁명과 같이 대부분의 내용은 우리가 어떻게 오늘날의 문명을 갖췄고 또 어떤 기술의 발견이 이를 이끌었는지에 관한 내용이 우리가 자주 마주하게 되는 인류 역사의 내용이다. 반면 데드먼드 모리스는 ‘털 없는 원숭이’를 통해서 인간의 문명이 아니라 인간 그 자체에 대해 관찰한 결과를 이야기한다. 인간을 지칭하는 ‘털 없는 원숭이’라는 용어에서부터 그가 어떤 방식으로 인류에 대해 말하고 싶어 하는지를 알 수 있다. 서론에서 말하듯, 우리가 ‘검은 발 다람쥐’ 에 대해 조사하고 싶다면 우선적으로 하는 일은 유사한 다른 여러 다람쥐 종들과는 구별되는 특성을 찾는 것이다. 검은 발 다람쥐의 구별되는 특성이 ‘검은 발’인 것처럼 그는 인간을 원숭이나 침팬지와 같은 유사한 영장류와 구분하여 ‘털 없는 원숭이’로서 객관적으로 인간을 바라보고자 한다. 적나라하게 분석되는 털 없는 원숭이에 대해 읽다보면 인간으로서 부정하고 싶은 충동이 들게 되지만, 논리적으로 관찰되고 전개되는 내용을 읽다보면 쉽사리 부정할 수 없을 만큼 스스로에 대해 새로운 시각을 갖게 된다.   8개의 챕터로 이루어진 책에서 가장 인상 깊은 챕터를 꼽는다면 단연 1챕터와 2챕터일 것이다. 1챕터 ‘기원’에서는 오늘날의 인간 호모 사피엔

In [96]:
import pickle

with open("doc_list.pkl","wb") as f:
    pickle.dump(doc_list, f)

In [97]:
import pickle

with open("doc_list.pkl","rb") as f:
    doc_list_load = pickle.load(f)

In [99]:
listLen = 0
for i in doc_list_load:
    listLen += len(i[1])
listLen / len(doc_list_load)

2912.3

In [101]:
loaded_model_all_mpnet_base_v2 = joblib.load('./kw_model_all-mpnet-base-v2.pkl')

In [102]:
loaded_model_base = joblib.load('./kw_model_base.pkl')

In [103]:
loaded_model_distiluse_base_multilingual_cased_v1 = joblib.load('./kw_model_distiluse-base-multilingual-cased-v1.pkl')

In [115]:
loaded_model_klue_bert_base = joblib.load('./kw_model_klue_bert-base.pkl')

In [105]:
loaded_model_klue_roberta_base = joblib.load('./kw_model_klue_roberta-base.pkl')

In [106]:
loaded_model_klue_roberta_small = joblib.load('./kw_model_klue_roberta-small.pkl')

In [107]:
loaded_model_klue_roberta_large = joblib.load('./kw_model_klue_roberta-large.pkl')

In [111]:
len(doc_list_load[4][1])

12145

In [112]:
len(doc_list_load[5][1])

412

In [109]:
for i in doc_list_load:
    keyBERT_model_whole(loaded_model_base, i[1])

[('생활환경', 0.4338), ('재구성한다', 0.4202), ('매력적이다', 0.4059), ('객관적으로', 0.3983), ('인간의', 0.3976)]
keywords_1: 0.41116
[('관점에서 재구성한다', 0.4717), ('역사의 내용이다', 0.465), ('불과함을 적나라하게', 0.462), ('생활환경 변화의', 0.4619), ('수백만 전의', 0.4603)]
keywords_2: 0.46418
[('독자의 상상력을 자극한다', 0.5061), ('역사는 수백만 전의', 0.5048), ('독특한 관점에서 재구성한다', 0.5038), ('종들과는 구별되는 특성을', 0.4951), ('육식을 하는 동물의', 0.4937)]
keywords_3: 0.5006999999999999
13.1042 sec
[('성균관대학교에서', 0.5298), ('천재과학자의', 0.5279), ('성균관대학교', 0.5189), ('일상생활의', 0.5139), ('문과학생들을', 0.5131)]
keywords_1: 0.5207200000000001
[('천재과학자의 현학적인', 0.6098), ('작동하는 방식과', 0.5667), ('성균관대학교에서 일어날', 0.5592), ('면모와 과학자로서의', 0.5578), ('과학자의 이미지와는', 0.5488)]
keywords_2: 0.56846
[('천재과학자의 현학적인 대화인', 0.6638), ('인간적인 면모와 과학자로서의', 0.6315), ('과학의 역할과 의미를', 0.6241), ('속에서 과학의 역할과', 0.6101), ('정치가 작동하는 방식과', 0.6092)]
keywords_3: 0.62774
15.2764 sec
[('국어국문학과', 0.489), ('형이상학과', 0.4846), ('공리주의의', 0.4559), ('공리주의적', 0.4548), ('의도성을', 0.4432)]
keywords_1: 0.4655
[('국어국문학과 장재영', 0.6396), (

In [110]:
for i in doc_list_load:
    keyBERT_model_whole(loaded_model_all_mpnet_base_v2, i[1])

[('역사부문에서', 0.6974), ('우선적으로', 0.692), ('재구성한다', 0.6831), ('문명들에', 0.6775), ('중요하지만', 0.6696)]
keywords_1: 0.68392
[('내용은 우리가', 0.7386), ('동시에 주제가', 0.7366), ('하나하나에 숨겨져', 0.7323), ('우선적으로 하는', 0.7306), ('우리의 기원에', 0.7279)]
keywords_2: 0.7332
[('내용이 우리가 자주', 0.7529), ('이유는 우리가 얼마나', 0.7495), ('동시에 주제가 주제인', 0.7489), ('우리가 다른 영장류와', 0.7473), ('우리는 우리 자신에', 0.7459)]
keywords_3: 0.7489000000000001
68.9312 sec
[('문과학생들을', 0.655), ('전문지식을', 0.6534), ('수식만큼이나', 0.6502), ('인간적인', 0.6269), ('비추어본다면', 0.6138)]
keywords_1: 0.63986
[('조국의 재건을', 0.7039), ('숙고와 고심하는', 0.6789), ('본인의 연구', 0.677), ('중심으로 움직인다', 0.677), ('무기화에는 적극', 0.6746)]
keywords_2: 0.68228
[('조국의 재건을 위해', 0.7097), ('이후 조국의 재건을', 0.7029), ('후에는 무기화에는 적극', 0.6945), ('수식과 실험에만 몰두하는', 0.6942), ('아니고 그의 수식과', 0.6909)]
keywords_3: 0.69844
76.2360 sec
[('운동에는', 0.6628), ('연구들에서', 0.6476), ('대한민국의', 0.6434), ('분석하는데', 0.6409), ('인문학적', 0.6389)]
keywords_1: 0.64672
[('수학의 흔적은', 0.6828), ('밴담은 운동에는', 0.6791), ('연구들에서 과학이나', 0.6788), ('영국의 

In [114]:
for i in doc_list_load:
    keyBERT_model_whole(loaded_model_distiluse_base_multilingual_cased_v1, i[1])

[('역사는', 0.3598), ('역사적', 0.3527), ('역사이다', 0.3518), ('역사의', 0.349), ('역사를', 0.3423)]
keywords_1: 0.35112
[('역사적 과학적', 0.3876), ('흔히 역사를', 0.3512), ('역사부문에서 두드러지게', 0.3426), ('역사를 통해', 0.3415), ('역사이다 기나긴', 0.3409)]
keywords_2: 0.35275999999999996
[('역사를 통해 배운다', 0.3836), ('역사이다 기나긴 역사', 0.3783), ('단순한 암기보다는 전후', 0.3751), ('암기보다는 전후 관계를', 0.3651), ('이해하고 미래까지 엿볼', 0.3541)]
keywords_3: 0.37124
8.8144 sec
[('성균관대학교에서', 0.4947), ('성균관대학교', 0.4564), ('성균관대에서', 0.3717), ('캠퍼스가', 0.2829), ('이과학생들이', 0.2375)]
keywords_1: 0.36863999999999997
[('성균관대학교에서 일어날', 0.5167), ('있는 성균관대학교에서', 0.4899), ('비단 성균관대학교', 0.442), ('성균관대학교 뿐만', 0.4398), ('성균관대에서 개최된', 0.3887)]
keywords_2: 0.45542
[('성균관대학교에서 일어날 법한', 0.5311), ('있는 성균관대학교에서 일어날', 0.5127), ('되어 있는 성균관대학교에서', 0.4717), ('이는 비단 성균관대학교', 0.4475), ('비단 성균관대학교 뿐만', 0.4355)]
keywords_3: 0.47970000000000007
9.6658 sec
[('도덕과', 0.4239), ('철학적으로', 0.3802), ('철학적', 0.3768), ('철학', 0.3579), ('원칙을', 0.3537)]
keywords_1: 0.3785
[('도덕과 입법의', 0.5471), ('장재영 도덕과

In [116]:
for i in doc_list_load:
    keyBERT_model_whole(loaded_model_klue_bert_base, i[1])

[('숲속에', 0.9617), ('숲에', 0.9605), ('있다', 0.9597), ('생각했던', 0.9596), ('싶은', 0.9596)]
keywords_1: 0.96022
[('재미있었다 저자는', 0.9662), ('때문이다 인간은', 0.9637), ('매력적이다 곤충을', 0.9635), ('것이다 그러나', 0.9628), ('영장류 숲속에', 0.9627)]
keywords_2: 0.9637800000000001
[('동시에 재미있었다 저자는', 0.9685), ('재미있었다 저자는 자신이', 0.9672), ('되기 때문이다 인간은', 0.9667), ('나타나는 특성이다 우리는', 0.9663), ('하는지를 있다 서론에서', 0.966)]
keywords_3: 0.9669399999999999
271.3007 sec
[('철학이라는', 0.9679), ('철학이었음을', 0.9675), ('철학에', 0.9674), ('철학적인', 0.9671), ('것이었다', 0.9669)]
keywords_1: 0.96736
[('철학적인 내용이', 0.9701), ('철학이라는 틀에', 0.9693), ('속에서 과학의', 0.969), ('철학이 현대사회와', 0.969), ('학문이라고 생각하는', 0.969)]
keywords_2: 0.96928
[('인식되고 있는 철학이', 0.971), ('호기심을 철학이라는 틀에', 0.9703), ('것이기도 하였다 실험실에', 0.9703), ('철학이라는 틀에 맞춰', 0.9699), ('일이다 역시 생활속의', 0.9699)]
keywords_3: 0.97028
307.4745 sec
[('미시적이고', 0.9682), ('것이었지만', 0.9679), ('있었으나', 0.9675), ('철학적', 0.9675), ('공리주의적', 0.9675)]
keywords_1: 0.9677200000000001
[('사상이었다는 또한', 0.9698), ('이유이다 나에게', 0.9697), ('책

In [138]:
for i in doc_list_load:
    keyBERT_model_whole(loaded_model_klue_roberta_base, i[1])

[('암기보다는', 0.9742), ('매력적이다', 0.9739), ('라는', 0.9731), ('보여주며', 0.973), ('것이다', 0.973)]
keywords_1: 0.9734399999999999
[('것이다 그러나', 0.9771), ('잡은 우리의', 0.9752), ('있다 나아가면', 0.975), ('것이다 오늘날', 0.975), ('합리적인 생활환경', 0.9749)]
keywords_2: 0.9754400000000001
[('것이다 그러나 충분히', 0.9784), ('대상이었다 그러나 우리는', 0.9778), ('것이다 오늘날 우리가', 0.9775), ('통해 배운다 라는', 0.9765), ('통해 바라볼 있다', 0.9764)]
keywords_3: 0.9773200000000001
81.0641 sec
[('철학이라는', 0.9648), ('일상생활', 0.9645), ('양자역학', 0.9645), ('수식만큼이나', 0.9644), ('양자역학을', 0.9644)]
keywords_1: 0.96452
[('것이었다 과학자는', 0.9684), ('것이었다 생활속의', 0.9683), ('것이다 하지만', 0.9673), ('성균관대학교 뿐만', 0.9671), ('통해 양자역학', 0.967)]
keywords_2: 0.9676199999999999
[('것이었다 생활속의 양자', 0.9708), ('생각했다 하지만 수업과', 0.9701), ('것이었다 과학자는 일상생활에서', 0.9697), ('인식시켜 것이었다 과학자는', 0.9695), ('어렵다는 것이다 하지만', 0.9691)]
keywords_3: 0.9698399999999999
90.9583 sec
[('공리주의란', 0.9662), ('공리주의로', 0.966), ('8장', 0.9657), ('공리주의라는', 0.9653), ('것은', 0.9648)]
keywords_1: 0.9656
[('과학 이라는', 0.9674), ('것이다 그는', 

In [137]:
for i in doc_list_load:
    keyBERT_model_whole(loaded_model_klue_roberta_small, i[1])

[('2챕터', 0.812), ('2챕터일', 0.812), ('1챕터', 0.8091), ('1챕터와', 0.8091), ('에서는', 0.8059)]
keywords_1: 0.80962
[('기대감이 깃들어있다', 0.8446), ('말해주고 있다', 0.8411), ('간과되어서는 것이다', 0.8393), ('이야기한다 인간을', 0.8382), ('관점에서 재구성한다', 0.8352)]
keywords_2: 0.8396800000000001
[('인간을 바라보고자 한다', 0.8745), ('이야기한다 인간을 지칭하는', 0.8647), ('현생인류의 등장에 대해', 0.8597), ('나타나는 특성이다 우리는', 0.8573), ('역사부문에서 두드러지게 나타나는', 0.8564)]
keywords_3: 0.86252
41.3753 sec
[('양자역학은', 0.7861), ('느끼곤한다', 0.7847), ('상대성이론을', 0.7829), ('사유한다는', 0.7818), ('정치인에게서도', 0.7801)]
keywords_1: 0.78312
[('개념을 활용한다는', 0.8382), ('비하하는 내용이라는', 0.8332), ('다루는 양자역학은', 0.832), ('천재과학자의 현학적인', 0.8304), ('숙고하였다 하이젠베르크는', 0.8258)]
keywords_2: 0.83192
[('문과학생들을 비하하는 내용이라는', 0.8619), ('개념을 바탕으로 사유한다는', 0.8615), ('철학 개념을 활용한다는', 0.856), ('세계를 다루는 양자역학은', 0.8534), ('개념을 활용한다는 것과', 0.8466)]
keywords_3: 0.8558800000000002
43.5313 sec
[('항목화하고자', 0.7877), ('비실증적인', 0.7835), ('경험주의라는', 0.7827), ('222p에서부터는', 0.7778), ('실현하고자', 0.7774)]
keywords_1: 0.78182
[('입법과 과학이라

In [139]:
for i in doc_list_load:
    keyBERT_model_whole(loaded_model_klue_roberta_large, i[1])

[('숲속에', 0.9617), ('숲에', 0.9605), ('있다', 0.9597), ('생각했던', 0.9596), ('싶은', 0.9596)]
keywords_1: 0.96022
[('재미있었다 저자는', 0.9662), ('때문이다 인간은', 0.9637), ('매력적이다 곤충을', 0.9635), ('것이다 그러나', 0.9628), ('영장류 숲속에', 0.9627)]
keywords_2: 0.9637800000000001
[('동시에 재미있었다 저자는', 0.9685), ('재미있었다 저자는 자신이', 0.9672), ('되기 때문이다 인간은', 0.9667), ('나타나는 특성이다 우리는', 0.9663), ('하는지를 있다 서론에서', 0.966)]
keywords_3: 0.9669399999999999
295.7973 sec
[('철학이라는', 0.9679), ('철학이었음을', 0.9675), ('철학에', 0.9674), ('철학적인', 0.9671), ('것이었다', 0.9669)]
keywords_1: 0.96736
[('철학적인 내용이', 0.9701), ('철학이라는 틀에', 0.9693), ('속에서 과학의', 0.969), ('철학이 현대사회와', 0.969), ('학문이라고 생각하는', 0.969)]
keywords_2: 0.96928
[('인식되고 있는 철학이', 0.971), ('호기심을 철학이라는 틀에', 0.9703), ('것이기도 하였다 실험실에', 0.9703), ('철학이라는 틀에 맞춰', 0.9699), ('일이다 역시 생활속의', 0.9699)]
keywords_3: 0.97028
319.2193 sec
[('미시적이고', 0.9682), ('것이었지만', 0.9679), ('있었으나', 0.9675), ('철학적', 0.9675), ('공리주의적', 0.9675)]
keywords_1: 0.9677200000000001
[('사상이었다는 또한', 0.9698), ('이유이다 나에게', 0.9697), ('책

In [69]:
bert_b = TransformerDocumentEmbeddings('klue/roberta-base')
total_start = time.time()
for i in doc_list:
    print(data['title'][i[0]])
    keyBERT_model(bert_b, i[1])
print(f"{time.time()-total_start:.4f} sec") # 수행시간 측정

바르톨로메는 개가 아니다 (라헐 판 코에이 장편소설)
[('양철', 0.9687), ('것은', 0.9685), ('와타나베에게서', 0.9684), ('숨쉬었다는', 0.9682), ('나오코란', 0.9682)]
[('앉아있었다 1982년에', 0.9721), ('떠나보냈다 와타나베의', 0.9709), ('그것을 찾았다', 0.9702), ('허무하지 않은가', 0.9701), ('죽음은 그것의', 0.9699)]
[('앉아있었다 1982년에 자신을', 0.9729), ('그것을 찾았다 삶은', 0.9727), ('좌석에 앉아있었다 1982년에', 0.9721), ('관계에서 표류하는 와타나베에게', 0.9718), ('나오코의 남자친구였다 나오코는', 0.9718)]
104.4026 sec
글로벌 거지 부부 (국적 초월, 나이 초월, 상식 초월, 9살 연상연하 커플의 무일푼 여행기)
[('이었습니다제가', 0.9748), ('것은', 0.9736), ('같았습니다', 0.9732), ('흥분되었습니다', 0.9731), ('문제라는', 0.973)]
[('충격 이었습니다제가', 0.9771), ('같습니다 저자는', 0.9768), ('않습니다 저자는', 0.9767), ('되었습니다 한마디로', 0.9754), ('것입니다 인간의', 0.9753)]
[('같습니다 저자는 인생이', 0.9783), ('느낌은 충격 이었습니다제가', 0.9782), ('그렇습니다 역행자로서 살아가며', 0.9777), ('충격 이었습니다제가 책을', 0.9777), ('책인 같습니다 저자는', 0.9776)]
53.2123 sec
미스 플라이트 (박민정 장편소설)
[('같다', 0.9641), ('문장들만으로도', 0.964), ('주옥같은', 0.9637), ('아니다', 0.9637), ('이야기다', 0.9636)]
[('정세랑이었다 시선으로부터', 0.9676), ('감동했다 주옥같은', 0.9673), ('듯하다 지난', 0.9663), ('나에게 학기는', 0

In [8]:
!pip install kiwipiepy

In [113]:
from kiwipiepy import Kiwi
kiwi = Kiwi()

In [12]:
tokenized_doc = kiwi.analyze(doc_list[0][1])
#명사 뽑기
tokenized_nouns = ' '.join([i[0] for word in tokenized_doc for i in word[0] if i[1].startswith('NN')])
tokenized_nouns

'그때 보잉 기기 좌석 년 자신 회고 와타나베 독백 자신 속마음 고백 듯 어조 이야기 시작 과거 초원 와타나베 곁 나오코 여자 나오코 자신 그림자 고백 도로시 오즈 마법사 양철 인간 공허감 와타나베 희망 말 한마디 희망 고문 수 자신 갈구 이 자신 기억 부탁 자신 세상 숨 사실 세상 일부 존재 것 것 기억 시간 나오코 죽음 이후 와타나베 시작 단순 사랑 주제 이야기 요약 것 소설 핵심 생 사 연속 때문 소설 전반 주인공 결여 삶 와타나베 기숙사 선배 나가 사 도덕 결여 삶 점 눈 점 와타나베 지옥 삶 생각 나오코 정신 문제 요양소 레이코 음모 남편 딸 결별 자신 아이 주인공 와타나베 나오코 살 기즈키 아이 와타나베 절친 나오코 남자 친구 나오코 이후 말 때 자신 느낌 말 것 정신 문제 결국 생 마감 와타나베 기즈키 나오코 소설 무결 사람 세상 사람 극복 사람 존재 뿐 소설 사람 미도리 미도리 와타나베 대학교 학우 날 와타나베 음식점 미도리 와타나베 말투 호감 감정 발전 인상 모습 미도리 그림자 어머니 나이 세상 아버지 딸 관심 미도리 앞 나오코 관계 표류 와타나베 닻 사랑 나오코 수 용기 복 작가 말 와타나베 나오코 미도리 사람 채 어찌저찌 세상 것 진심 사랑 이 존재 소멸 과정 절망 죽음 확정 사실 삶 속 죽음 대극 일부 존재 곁 도움 기즈키 와타나베 나오코 레이코 요양소 사람 도우 것 미도리 와타나베 삶 죽음 존재 죽음 매몰 그림자 곳 결국 자신 블랙홀 잠식 것 허무 필자 이야기 처음 것 재수 학원 자습 실 허리 고개 주위 모두 석상 책상 책 공부 수 숨 책 학원 너머 나오코 초원 상상 산들바람 억새풀 몸 나오코 머릿결 책장 나오코 소설 속 죽음 등장인물 결과 것 활자 존재 책 때 말 자신 행간 숨 자신 이야기 때 번 책장 수 행복 경험 것 터 격변 현대 사회 정적 텍스트 역할 잠 고요 새벽 시간 하루키 노르웨이 숲 나오코 브람스 교향곡 번 우울 인생 서로 의지 앞 것 서로 서로 존재 것 기억'

In [ ]:
def noun_tokenize(doc):
    tokenized_doc = kiwi.analyze(doc)
    tokenized_nouns = ' '.join([i[0] for word in tokenized_doc for i in word[0] if i[1].startswith('NN')])
    return tokenized_nouns

In [127]:
def keyBERT_model_noun(modelname, doc):
    start = time.time()
    kw_model = KeyBERT(model=modelname)
    tokenized_doc = kiwi.analyze(doc)
    tokenized_nouns = ' '.join([i[0] for word in tokenized_doc for i in word[0] if i[1].startswith('NN')])
    keywords_1 = kw_model.extract_keywords(tokenized_nouns, keyphrase_ngram_range=(1, 1), stop_words=None)
    keywords_2 = kw_model.extract_keywords(tokenized_nouns, keyphrase_ngram_range=(2, 2), stop_words=None)
    keywords_3 = kw_model.extract_keywords(tokenized_nouns, keyphrase_ngram_range=(3, 3), stop_words=None)
    print(keywords_1)
    print(keywords_2)
    print(keywords_3)
    print(f"{time.time()-start:.4f} sec") # 수행시간 측정

In [15]:
keyBERT_model(roberta_s, tokenized_nouns)

[('와타나베', 0.7425), ('나오코', 0.7285), ('무결', 0.7222), ('허무', 0.7128), ('레이코', 0.712)]
[('와타나베 사랑', 0.7863), ('와타나베 음식점', 0.7822), ('와타나베 나오코', 0.7762), ('와타나베 죽음', 0.7725), ('와타나베 희망', 0.7724)]
[('표류 와타나베 사랑', 0.7977), ('회고 와타나베 독백', 0.796), ('미도리 와타나베 죽음', 0.7917), ('공허감 와타나베 희망', 0.7908), ('와타나베 사랑 나오코', 0.7878)]
22.7872 sec


In [16]:
roberta_s = TransformerDocumentEmbeddings('klue/roberta-small')
total_start = time.time()
for i in doc_list:
    print(data['title'][i[0]])
    keyBERT_model_noun(roberta_s, i[1])
    print()
print(f"{time.time()-total_start:.4f} sec") # 수행시간 측정

복덕방
[('와타나베', 0.7425), ('나오코', 0.7285), ('무결', 0.7222), ('허무', 0.7128), ('레이코', 0.712)]
[('와타나베 사랑', 0.7863), ('와타나베 음식점', 0.7822), ('와타나베 나오코', 0.7762), ('와타나베 죽음', 0.7725), ('와타나베 희망', 0.7724)]
[('표류 와타나베 사랑', 0.7977), ('회고 와타나베 독백', 0.796), ('미도리 와타나베 죽음', 0.7917), ('공허감 와타나베 희망', 0.7908), ('와타나베 사랑 나오코', 0.7878)]
22.7102 sec

한국의 능력주의 (한국인이 기꺼이 참거나 죽어도 못 참는 것에 대하여)
[('솓구', 0.7486), ('순리자', 0.7363), ('안정감', 0.7237), ('이것저것', 0.7192), ('작정', 0.7187)]
[('마음 솓구', 0.777), ('창피 순리자', 0.7636), ('솓구 모습', 0.7555), ('순리자 모습', 0.7529), ('설명 얘기', 0.7527)]
[('인정 마음 솓구', 0.7894), ('순리자 모습 이야기', 0.7884), ('얘기 창피 순리자', 0.7857), ('순리 세상 설명', 0.7805), ('자신 일시 안정감', 0.7799)]
9.5780 sec

모든 관계는 말투에서 시작된다(10만 부 기념 스페셜 에디션) (기분 좋은 사람으로 기억되는 사소한 습관)
[('상태', 0.7265), ('해파리', 0.7212), ('박서련', 0.7137), ('기대', 0.7095), ('실질', 0.7083)]
[('텔레파시 생각', 0.7503), ('주옥 구절', 0.7458), ('정말 상태', 0.7386), ('시간 이번', 0.7349), ('기대 실망', 0.7328)]
[('박서련 박상영 백수', 0.7687), ('불안 마음 진통제', 0.7677), ('하늘 텔레파시 생각', 0.7594), ('마음 진

In [140]:
def keyBERT_model_noun_maxsum(modelname, doc):
    start = time.time()
    tokenized_doc = kiwi.analyze(doc)
    kw_model = KeyBERT(model=modelname)
    tokenized_nouns = ' '.join([i[0] for word in tokenized_doc for i in word[0] if i[1].startswith('NN')])
    keywords_1 = kw_model.extract_keywords(tokenized_nouns, keyphrase_ngram_range=(1, 1),use_maxsum=True, nr_candidates=20, stop_words=None)
    keywords_2 = kw_model.extract_keywords(tokenized_nouns, keyphrase_ngram_range=(2, 2),use_maxsum=True, nr_candidates=20, stop_words=None)
    keywords_3 = kw_model.extract_keywords(tokenized_nouns, keyphrase_ngram_range=(3, 3),use_maxsum=True, nr_candidates=20, stop_words=None)
    print(keywords_1)
    print(keywords_2)
    print(keywords_3)
    print(f"{time.time()-start:.4f} sec")

In [104]:
roberta_s = TransformerDocumentEmbeddings('klue/roberta-small')
total_start = time.time()
for i in doc_list:
    print(data['title'][i[0]])
    keyBERT_model_noun_maxsum(roberta_s, i[1])
    print()
print(f"{time.time()-total_start:.4f} sec") # 수행시간 측정


바르톨로메는 개가 아니다 (라헐 판 코에이 장편소설)
[('회고', 0.7038), ('처음', 0.7073), ('어찌저찌', 0.7101), ('공허감', 0.7116), ('와타나베', 0.7425)]
[('학원 자습', 0.7495), ('작가 와타나베', 0.7507), ('공허감 와타나베', 0.7626), ('와타나베 기즈키', 0.7628), ('와타나베 나오코', 0.7762)]
[('산들바람 억새풀 나오코', 0.7761), ('인간 공허감 와타나베', 0.7766), ('작가 와타나베 나오코', 0.7785), ('와타나베 음식점 미도리', 0.7842), ('공허감 와타나베 희망', 0.7908)]
24.0272 sec

글로벌 거지 부부 (국적 초월, 나이 초월, 상식 초월, 9살 연상연하 커플의 무일푼 여행기)
[('내용', 0.7009), ('소개', 0.7025), ('안정감', 0.7237), ('순리자', 0.7363), ('솓구', 0.7486)]
[('내용 내용', 0.7399), ('이상 순리자', 0.7406), ('순리자 자신', 0.7408), ('세상 설명', 0.7418), ('시간 투자', 0.7517)]
[('처음 한마디 느낌', 0.7673), ('안정감 자위 합리', 0.7695), ('발전 시간 투자', 0.7703), ('창피 순리자 모습', 0.7738), ('방법 설명 얘기', 0.7775)]
10.5168 sec

미스 플라이트 (박민정 장편소설)
[('공허', 0.6997), ('주말', 0.7), ('시점', 0.7006), ('박상영', 0.7037), ('덧니', 0.7079)]
[('공허 불안', 0.7229), ('결말 덧니', 0.7242), ('이야기 전개', 0.7245), ('일과 모멸감', 0.7292), ('주옥 구절', 0.7458)]
[('주말 일과 모멸감', 0.7528), ('정세랑 덧니 구절', 0.7544), ('공허 불안 마음', 0.7554), ('본가 혼자 현

In [129]:
def keyBERT_model_noun_mmr(modelname, doc):
    start = time.time()
    tokenized_doc = kiwi.analyze(doc)
    kw_model = KeyBERT(model=modelname)
    tokenized_nouns = ' '.join([i[0] for word in tokenized_doc for i in word[0] if i[1].startswith('NN')])
    keywords_1 = kw_model.extract_keywords(tokenized_nouns, keyphrase_ngram_range=(1, 1),use_mmr=True,diversity=0.7, stop_words=None)
    keywords_2 = kw_model.extract_keywords(tokenized_nouns, keyphrase_ngram_range=(2, 2),use_mmr=True,diversity=0.7, stop_words=None)
    keywords_3 = kw_model.extract_keywords(tokenized_nouns, keyphrase_ngram_range=(3, 3),use_mmr=True,diversity=0.7, stop_words=None)
    print(keywords_1)
    print(keywords_2)
    print(keywords_3)
    print(f"{time.time()-start:.4f} sec")

In [151]:
def keyBERT_model_noun_mmr_whole(model, doc):
    start = time.time()
    tokenized_doc = kiwi.analyze(doc)
    kw_model = model
    tokenized_nouns = ' '.join([i[0] for word in tokenized_doc for i in word[0] if i[1].startswith('NN')])
    keywords_1 = kw_model.extract_keywords(tokenized_nouns, keyphrase_ngram_range=(1, 1),use_mmr=True,diversity=0.3, stop_words=None)
    keywords_2 = kw_model.extract_keywords(tokenized_nouns, keyphrase_ngram_range=(2, 2),use_mmr=True,diversity=0.3, stop_words=None)
    keywords_3 = kw_model.extract_keywords(tokenized_nouns, keyphrase_ngram_range=(3, 3),use_mmr=True,diversity=0.3, stop_words=None)
    print(keywords_1)
    print(keywords_2)
    print(keywords_3)
    print(f"{time.time()-start:.4f} sec")

In [152]:
for i in doc_list_load:
    keyBERT_model_noun_mmr_whole(loaded_model_klue_roberta_small, i[1])

[('챕터', 0.7513), ('이성', 0.7202), ('만큼', 0.719), ('사피엔스', 0.7134), ('문명사회', 0.7024)]
[('자극 챕터', 0.7801), ('인상 챕터', 0.7796), ('짝짓기 챕터', 0.7749), ('사피엔스 등장', 0.7574), ('육지 영장류', 0.7454)]
[('상상력 자극 챕터', 0.7951), ('이해 관심 중요', 0.7813), ('인류 역사 관심', 0.7782), ('농업 혁명 과학', 0.7723), ('영장류 적응 동물', 0.768)]
20.2993 sec
[('하이젠베르크', 0.7203), ('중요', 0.7153), ('괴팅겐', 0.7127), ('패망', 0.7064), ('사유', 0.7022)]
[('하이젠베르크 양자', 0.7594), ('파급력 고려', 0.7444), ('인과율 개념', 0.7422), ('과학자 토론', 0.7395), ('일상생활 과학', 0.727)]
[('파급력 숙고 하이젠베르크', 0.7841), ('학문 유리 현상', 0.7831), ('원리 탐구 과학', 0.7825), ('하이젠베르크 철학 일상생활', 0.7778), ('하이젠베르크 양자 역학', 0.7697)]
25.9606 sec
[('국어국문학과', 0.7309), ('년대', 0.7284), ('경험주의', 0.727), ('이리가레이', 0.7257), ('공리주의', 0.7182)]
[('공리주의 공부', 0.78), ('공리주의 국어국문학과', 0.7749), ('국어국문학과 장재영', 0.7714), ('경험주의 주제', 0.7642), ('접근법 사용', 0.74)]
[('공리주의 국어국문학과 장재영', 0.8016), ('공리주의 개념 경우', 0.7979), ('경험주의 주제 아래', 0.797), ('공리주의 공부 사상', 0.7935), ('라캉 가타리 사상가', 0.7835)]
27.2213 sec
[('도리', 0.7032), ('중요', 0.70

In [143]:
for i in doc_list_load:
    keyBERT_model_noun_mmr_whole(loaded_model_klue_roberta_base, i[1])

[('때문', 0.9498), ('문명사회', 0.9497), ('사피엔스', 0.9475), ('과학', 0.9452), ('이상', 0.942)]
[('생활 환경', 0.9537), ('육지 영장류', 0.9525), ('상상력 자극', 0.9518), ('문명 인간', 0.9463), ('만큼 동물', 0.9385)]
[('반응 파악 중요', 0.9583), ('인간 부정 충동', 0.9546), ('경향 농업 혁명', 0.9545), ('영장류 적응 동물', 0.9539), ('동물 특성 육식', 0.9489)]
37.9475 sec
[('일상생활', 0.9523), ('이후', 0.9496), ('성균관대학교', 0.9493), ('과학', 0.9484), ('마흐', 0.9427)]
[('지식 과학', 0.9565), ('철학 현대', 0.9541), ('현대 사회', 0.9513), ('작동 방식', 0.9508), ('철학 일상생활', 0.9437)]
[('개최 체육 대회', 0.9615), ('정치 작동 방식', 0.9566), ('고민 고민 과학자', 0.9563), ('세계 구성 원리', 0.9548), ('시민 대량 살상', 0.9462)]
47.7340 sec
[('공리주의', 0.9495), ('국어국문학과', 0.947), ('접근법', 0.9444), ('장재영', 0.943), ('감수성', 0.9351)]
[('밴담 공리주의', 0.9517), ('기준 공리주의', 0.9505), ('참조 경우', 0.9501), ('현대 사회', 0.9473), ('이래 이리가레이', 0.9272)]
[('공리주의 입법 원칙', 0.956), ('경우 중학교 학년', 0.9555), ('기준 공리주의 관점', 0.954), ('행동 의도 방법', 0.9518), ('본인 사상 과학', 0.9388)]
50.1740 sec
[('번째', 0.9524), ('융화', 0.9512), ('때문', 0.9504), ('생명관', 0.9461), ('

In [144]:
for i in doc_list_load:
    keyBERT_model_noun_mmr_whole(loaded_model_klue_roberta_large, i[1])

[('문명사회', 0.9535), ('식충류', 0.9529), ('영장류', 0.9525), ('관심', 0.9519), ('사실', 0.9503)]
[('내용 반면', 0.957), ('챕터 짝짓기', 0.9553), ('문명 반응', 0.9553), ('인간 인간', 0.9527), ('곤충 식충류', 0.9527)]
[('성공 동물 생각', 0.9609), ('문명 인간 자체', 0.9598), ('매력 곤충 식충류', 0.9563), ('바다 유선 직립', 0.9556), ('챕터 챕터 챕터', 0.9468)]
122.8800 sec
[('과학', 0.9431), ('성균관대학교', 0.9403), ('성균관대', 0.9388), ('괴팅겐', 0.9383), ('당시', 0.9381)]
[('기술 방식', 0.9473), ('철학 일상생활', 0.9456), ('이원 성균관대학교', 0.944), ('고민 고민', 0.9398), ('얼마 성균관대', 0.9307)]
[('천재 과학자 현학', 0.9508), ('개념 활용 개념', 0.9476), ('모습 모습 인간', 0.9469), ('규합 연구소 개발', 0.944), ('성균관대학교 해프닝 성균관대학교', 0.9431)]
156.0866 sec
[('경험주의', 0.9431), ('이리가레이', 0.9392), ('단어', 0.9379), ('년대', 0.9376), ('당시', 0.9353)]
[('가능 공리주의', 0.9486), ('공리주의 체계', 0.948), ('분야 분야', 0.946), ('논의 운동', 0.9449), ('철학 사상', 0.9425)]
[('체계 사상 노력', 0.95), ('사용 학문 과학', 0.9497), ('원칙 서론 공리주의', 0.9493), ('대상 철학 사상', 0.9472), ('입법 해당 분야', 0.9446)]
164.7137 sec
[('번째', 0.9425), ('생명관', 0.942), ('문화', 0.9418), ('이유', 0.93

In [145]:
for i in doc_list_load:
    keyBERT_model_noun_mmr_whole(loaded_model_klue_bert_base, i[1])

[('문명사회', 0.9535), ('식충류', 0.9529), ('영장류', 0.9525), ('관심', 0.9519), ('사실', 0.9503)]
[('내용 반면', 0.957), ('챕터 짝짓기', 0.9553), ('문명 반응', 0.9553), ('인간 인간', 0.9527), ('곤충 식충류', 0.9527)]
[('성공 동물 생각', 0.9609), ('문명 인간 자체', 0.9598), ('매력 곤충 식충류', 0.9563), ('바다 유선 직립', 0.9556), ('챕터 챕터 챕터', 0.9468)]
123.6592 sec
[('과학', 0.9431), ('성균관대학교', 0.9403), ('성균관대', 0.9388), ('괴팅겐', 0.9383), ('당시', 0.9381)]
[('기술 방식', 0.9473), ('철학 일상생활', 0.9456), ('이원 성균관대학교', 0.944), ('고민 고민', 0.9398), ('얼마 성균관대', 0.9307)]
[('천재 과학자 현학', 0.9508), ('개념 활용 개념', 0.9476), ('모습 모습 인간', 0.9469), ('규합 연구소 개발', 0.944), ('성균관대학교 해프닝 성균관대학교', 0.9431)]
155.3813 sec
[('경험주의', 0.9431), ('이리가레이', 0.9392), ('단어', 0.9379), ('년대', 0.9376), ('당시', 0.9353)]
[('가능 공리주의', 0.9486), ('공리주의 체계', 0.948), ('분야 분야', 0.946), ('논의 운동', 0.9449), ('철학 사상', 0.9425)]
[('체계 사상 노력', 0.95), ('사용 학문 과학', 0.9497), ('원칙 서론 공리주의', 0.9493), ('대상 철학 사상', 0.9472), ('입법 해당 분야', 0.9446)]
164.7793 sec
[('번째', 0.9425), ('생명관', 0.942), ('문화', 0.9418), ('이유', 0.93

In [146]:
for i in doc_list_load:
    keyBERT_model_noun_mmr_whole(loaded_model_distiluse_base_multilingual_cased_v1, i[1])

[('역사', 0.313), ('원숭이', 0.168), ('논리', 0.1628), ('산업', 0.1112), ('기대감', 0.0323)]
[('혁명 과학', 0.4204), ('전후 관계', 0.2921), ('모리스 원숭이', 0.2265), ('내용 오늘날', 0.2066), ('도구 사용', -0.0457)]
[('혁명 과학 혁명', 0.4292), ('관계 파악 논리', 0.2971), ('기억 부분 예측', 0.266), ('데드먼드 모리스 원숭이', 0.2236), ('오늘날 발자취 현재', 0.2232)]
3.5899 sec
[('성균관대학교', 0.3703), ('공학', 0.1622), ('플래카드', 0.1122), ('변화', 0.0742), ('하이젠베르크', -0.0296)]
[('성균관대학교 해프닝', 0.3792), ('변화 기술', 0.2199), ('체육 대회', 0.1911), ('플래카드 문과', 0.1207), ('세계 두려움', 0.0401)]
[('성균관대학교 해프닝 성균관대학교', 0.4585), ('개최 체육 대회', 0.2174), ('학생 플래카드 문과', 0.2075), ('기술 방식 변화', 0.2064), ('대부분 천재 과학자', 0.1749)]
4.6012 sec
[('도덕', 0.4098), ('과학', 0.2224), ('국어국문학과', 0.1756), ('오늘날', 0.0113), ('움직임', -0.117)]
[('도덕 입법', 0.5195), ('국어국문학과 장재영', 0.1764), ('밴담 스튜어트', 0.0961), ('이름 아래', -0.0252), ('고립 분야', -0.1069)]
[('도덕 입법 원칙', 0.5817), ('단어 때문 저자', 0.13), ('목록 대응 동기', 0.0123), ('최대 다수 최대', 0.0113), ('학문 고립 분야', -0.0236)]
4.6379 sec
[('숭배', 0.2534), ('한국', 0.1857), ('조명', 0.088), 

In [147]:
for i in doc_list_load:
    keyBERT_model_noun_mmr_whole(loaded_model_base, i[1])

[('문명', 0.2903), ('절대', 0.2103), ('하나하나', 0.1493), ('경향', 0.0654), ('표현', 0.0373)]
[('인간 문명', 0.3805), ('육지 영장류', 0.2175), ('부분 예측', 0.1561), ('챕터 챕터', 0.1273), ('제시 다양', 0.1038)]
[('생활 환경 변화', 0.4562), ('특성 역사 오늘날', 0.3328), ('미래 기대감 미래', 0.2928), ('인류 역사 인류', 0.2024), ('챕터 짝짓기 챕터', 0.1436)]
3.0783 sec
[('성균관대학교', 0.5062), ('심리', 0.1239), ('부분', 0.1174), ('칸트', 0.1111), ('영향력', 0.0657)]
[('성균관대학교 해프닝', 0.6194), ('대화 대화', 0.2793), ('모습 모습', 0.2127), ('진리 추구', 0.1758), ('영향력 발휘', 0.0767)]
[('성균관대학교 해프닝 성균관대학교', 0.6406), ('기반 아인슈타인 하이젠베르크', 0.367), ('종합 지적 양자', 0.2512), ('사람 세계 두려움', 0.1866), ('사회 영향력 발휘', 0.0817)]
3.9982 sec
[('국어국문학과', 0.3434), ('정립', 0.1464), ('물체', 0.1201), ('오늘날', 0.044), ('뒷받침', 0.0188)]
[('공리주의 국어국문학과', 0.5493), ('유지 입법', 0.2009), ('쾌락 표현', 0.0775), ('이름 아래', 0.0579), ('분야 분야', 0.0306)]
[('공리주의 국어국문학과 장재영', 0.6262), ('행동 양식 정리', 0.2051), ('학문 도움 년대', 0.177), ('데카르트 홉스 칸트', 0.1527), ('다양 분야 분야', 0.0311)]
4.0529 sec
[('생명관', 0.4124), ('시비', 0.1286), ('하나', 0.0703), 

In [148]:
for i in doc_list_load:
    keyBERT_model_noun_mmr_whole(loaded_model_all_mpnet_base_v2, i[1])

[('원숭이', 0.5654), ('독특', 0.4707), ('미래', 0.3872), ('바다', 0.2782), ('다람쥐', 0.1441)]
[('분석 원숭이', 0.7523), ('통제 생각', 0.5901), ('반면 데드먼드', 0.5586), ('예측 수준', 0.5055), ('이해 미래', 0.4577)]
[('구분 원숭이 객관', 0.7767), ('만큼 스스로 시각', 0.6507), ('신비 동시 재미', 0.6126), ('챕터 챕터 챕터', 0.4821), ('이유 다양 이유', 0.4539)]
18.5794 sec
[('성균관대학교', 0.6776), ('적극', 0.499), ('플래카드', 0.4336), ('추구', 0.4048), ('영향력', 0.1576)]
[('숙고 하이젠베르크', 0.7588), ('성균관대학교 해프닝', 0.7268), ('체육 대회', 0.4587), ('대화 대화', 0.3934), ('붕괴 연쇄', 0.1354)]
[('성균관대학교 사회 학문', 0.8038), ('무기 직감 조국', 0.6212), ('개최 체육 대회', 0.5069), ('토로 모습 모습', 0.4743), ('사회 영향력 발휘', 0.1378)]
29.4096 sec
[('국어국문학과', 0.6542), ('포스트모더니즘', 0.5807), ('스튜어트', 0.4375), ('라캉', 0.3407), ('경계', 0.148)]
[('공리주의 국어국문학과', 0.7929), ('당시 데카르트', 0.5292), ('목록 미각', 0.453), ('체계 사상', 0.4356), ('분야 분야', 0.1196)]
[('공리주의 국어국문학과 장재영', 0.8133), ('이래 이리가레이 크리스테바', 0.5259), ('지점 예시 제시', 0.5113), ('목록 미각 감각', 0.4906), ('다양 분야 분야', 0.1214)]
31.5145 sec
[('긍정', 0.5866), ('순수', 0.4449), ('부모', 0.4

In [110]:
roberta_s = TransformerDocumentEmbeddings('klue/roberta-small')
total_start = time.time()
for i in doc_list:
    print(data['title'][i[0]])
    keyBERT_model_noun_mmr(roberta_s, i[1])
    print()
print(f"{time.time()-total_start:.4f} sec") # 수행시간 측정

바르톨로메는 개가 아니다 (라헐 판 코에이 장편소설)
[('와타나베', 0.7425), ('공허감', 0.7116), ('산들바람', 0.6982), ('마감', 0.6952), ('하루키', 0.6865)]
[('와타나베 사랑', 0.7863), ('학원 자습', 0.7495), ('산들바람 억새풀', 0.714), ('기즈키 나오코', 0.7089), ('요양소 사람', 0.6993)]
[('표류 와타나베 사랑', 0.7977), ('재수 학원 자습', 0.7518), ('나오코 여자 나오코', 0.7444), ('죽음 존재 죽음', 0.7311), ('상상 산들바람 억새풀', 0.7152)]
23.8927 sec

글로벌 거지 부부 (국적 초월, 나이 초월, 상식 초월, 9살 연상연하 커플의 무일푼 여행기)
[('솓구', 0.7486), ('순리자', 0.7363), ('안정감', 0.7237), ('설명', 0.6959), ('한마디', 0.674)]
[('마음 솓구', 0.777), ('순리자 자신', 0.7408), ('인간 본성', 0.7148), ('행복 인간', 0.7017), ('성공 방법', 0.6886)]
[('인정 마음 솓구', 0.7894), ('경제 시간 자유', 0.7578), ('플루 추천 도서', 0.7554), ('행복 인간 본성', 0.7402), ('사회 성공 방법', 0.728)]
9.8814 sec

미스 플라이트 (박민정 장편소설)
[('상태', 0.7265), ('텔레파시', 0.6957), ('모멸감', 0.679), ('본가', 0.6779), ('이틀간', 0.6606)]
[('텔레파시 생각', 0.7503), ('이야기 전개', 0.7245), ('정세랑 덧니', 0.7076), ('중반부 구절', 0.7021), ('구절 인스타', 0.6897)]
[('박서련 박상영 백수', 0.7687), ('주말 일과 모멸감', 0.7528), ('해파리 언니 정세랑', 0.7214), ('회충 혈관 사이', 0.704

In [130]:
def choiceExtract(docData):
    print('키워드 추출을 진행하시겠습니까? y/n')
    choice = input();
    if(choice.lower() == 'y'):
        try:
            print("키워드 추출 중... 10~20여초 가량 소요됩니다.")
            keyBERT_model_noun_mmr('klue/roberta-small', docData['report_text'])
        except extractError:
            print("Extract Error")

In [131]:
def printDetail(data):
    print('세부사항을 확인할 데이터 열의 인덱스를 입력하세요')
    idx = int(input())
    try:
        print(data.loc[idx])
        choiceExtract(data.loc[idx])
    except KeyError:
        print("Key Error")

In [132]:
import datetime
def insertData(data):
    print('추가할 독서 리뷰 저자를 입력하세요.')
    writer = input()
    print('추가할 도서 제목을 입력하세요')
    title = input()
    print('도서 저자를 입력하세요')
    book_writer = input()
    print('출판사를 입력하세요')
    company = input()
    print('도서 출간일을 입력하세요')
    book_date = input()
    print('독서 리뷰 본문을 입력하세요')
    report_text = input()
    now = datetime.datetime.now()
    formattedDate = now.strftime("%Y/%m/%d")
    newData = pd.DataFrame({'report_writer' : [writer], 'date_writer':[formattedDate],'title' : [title], 'book_writer' : [book_writer], 'book_company':[company],
                       'book_date':[book_date], 'report_text':[report_text]})
    try:
        data = pd.concat([data,newData], ignore_index = True)
        print("입력 성공")
        print(data.loc[len(data)-1])
        choiceExtract(data.loc[len(data)-1])
    except insertErr:
        print("insertErr")
    


In [133]:
def searchTitle(data):
    print('1. 제목 검색/키워드 추출\n')
    print('키워드를 추출할 도서 제목을 입력해주세요.')
    title = input()
    df = data[data['title'].str.contains(title)]
    print('검색된 데이터 개수',len(df))
    print(df[['title', 'report_writer', 'report_text']])
    printDetail(data)


In [134]:
def menu():
    print(""" -------------------------------------
수행할 작업 입력
1. 제목 검색/키워드 추출
2. 신규 문서 추가/키워드 추출
3. 자료 id로 검색/키워드 추출
4. 이외 입력. 종료
-------------------------------------""")
    user_in = input()
    if(user_in == '1'):
        searchTitle(data)
        return True;
    elif(user_in == '2'):
        print('2. 신규 문서 추가/키워드 추출\n')
        insertData(data)
        return True;
    elif(user_in == '3'):
        printDetail(data)
        return True;
    else:
        print('이외 입력. 종료')
        return False;
    

In [117]:
chk = True
while(chk):
    chk = menu()

 -------------------------------------
수행할 작업 입력
1. 제목 검색/키워드 추출
2. 신규 문서 추가/키워드 추출
3. 자료 id로 검색/키워드 추출
4. 이외 입력. 종료
-------------------------------------
2
2. 신규 문서 추가/키워드 추출

추가할 독서 리뷰 저자를 입력하세요.
지구인
추가할 도서 제목을 입력하세요
우주인
도서 저자를 입력하세요
태양인
출판사를 입력하세요
외계인
도서 출간일을 입력하세요
2022/12/23
독서 리뷰 본문을 입력하세요
봄누 머ㅓ어
입력 성공
report_writer           지구인
date_writer      2023/09/28
title                   우주인
book_writer             태양인
book_company            외계인
book_date        2022/12/23
report_text          봄누 머ㅓ어
Name: 1657, dtype: object
키워드 추출을 진행하시겠습니까? y/n
n
 -------------------------------------
수행할 작업 입력
1. 제목 검색/키워드 추출
2. 신규 문서 추가/키워드 추출
3. 자료 id로 검색/키워드 추출
4. 이외 입력. 종료
-------------------------------------
4
이외 입력. 종료


In [136]:
chk = True
while(chk):
    chk = menu()

 -------------------------------------
수행할 작업 입력
1. 제목 검색/키워드 추출
2. 신규 문서 추가/키워드 추출
3. 자료 id로 검색/키워드 추출
4. 이외 입력. 종료
-------------------------------------
1
1. 제목 검색/키워드 추출

키워드를 추출할 도서 제목을 입력해주세요.
노르웨이의 숲
검색된 데이터 개수 22
                              title             report_writer  \
0                           노르웨이의 숲                       조장환   
81                          노르웨이의 숲                       김가영   
262                         노르웨이의 숲                       zen   
1322                        노르웨이의 숲                        휴고   
1442            상실의 시대(원제: 노르웨이의 숲)  유생 선배님들은 귀차니즘을 어떻게 이겨냈을까   
1892            상실의 시대 (원제 노르웨이의 숲)                        다나   
2721                        노르웨이의 숲                        이담   
2983            상실의 시대 (원제 노르웨이의 숲)                        스지   
3337   노르웨이의 숲 (세계문학전집 310,ノルウェイの森)                    이승현123   
3469                        노르웨이의 숲                       좋은비   
4171                        노르웨이의 숲                       이경아   


No sentence-transformers model found with name C:\Users\liber/.cache\torch\sentence_transformers\klue_roberta-small. Creating a new one with MEAN pooling.


[('와타나베', 0.4462), ('공중전화', 0.4384), ('대극', 0.4192), ('기즈키', 0.3883), ('노르웨이', 0.3674)]
[('사람 사이', 0.5022), ('와타나베 나오코', 0.4955), ('공중전화 부스', 0.4656), ('이야기 의미', 0.4646), ('기즈키 연결', 0.4235)]
[('와타나베 자신 구절', 0.5446), ('풍경 따위 자신', 0.5263), ('등장인물 중심 이야기', 0.518), ('조차 사람 사이', 0.4975), ('공중전화 부스 주변', 0.4797)]
7.3877 sec
 -------------------------------------
수행할 작업 입력
1. 제목 검색/키워드 추출
2. 신규 문서 추가/키워드 추출
3. 자료 id로 검색/키워드 추출
4. 이외 입력. 종료
-------------------------------------
4
이외 입력. 종료
